# Read the preprocessed Data

In [1]:
import numpy as np
import pandas as pd

df=pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv",index_col=0)
df.head()

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,5,1,4.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
2,7,1,4.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
3,15,1,2.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
4,17,1,4.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995


# Collaborative Filtering with unsupervised algorithms with sklearn.neighbors

## 1. Create a Pivot Matrix

In [2]:
df_movie_pivot = df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

df_movie_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Convert into an Array Matrix by using csr_matrix

In [3]:
from scipy.sparse import csr_matrix
feature_matrix=csr_matrix(df_movie_pivot.values)

## 3. sklearn.neighbors

In [4]:

from sklearn.neighbors import NearestNeighbors

In [5]:
model_knn=NearestNeighbors(metric="cosine",algorithm="auto")
model_knn.fit(feature_matrix)

NearestNeighbors(metric='cosine')

In [6]:
movies=df['title'].unique()


In [7]:
def recommendation_movie():
    query_title = input('Enter your Facourite Movie: ')
    n= 0
    while query_title not in movies and n<5:# maximum 5 prompts
        print("The Movie ", query_title, "does not exist in our database.") 
        query_title = input('Please enter another Facourite Movie: ')
        n +=1
    #Find the movie ID w.r.t the the title
    query_index=df[df['title']==query_title]['movieId'].unique()[0]
    
    n_neigh=10
    distances,indices=model_knn.kneighbors(
        df_movie_pivot.iloc[query_index,:].values.reshape(1,-1),n_neighbors=n_neigh
    )
    #print("Distances --&gt;",distances," Indices --&gt;",indices)
 
    #print(distances.flatten())
    #print(len(distances.flatten()))
    
    for i in range(0,len(distances.flatten())):
        ii=i
        movieName=df[df['movieId']==df_movie_pivot.index[indices.flatten()[i]]]['title'].unique()[0]
        movieYear=df[df['movieId']==df_movie_pivot.index[indices.flatten()[i]]]['year'].unique()[0]
        match=(1-distances.flatten()[i]/1.)*100
        avgRat=df[df['movieId']==df_movie_pivot.index[indices.flatten()[i]]]['rating'].mean()
        ratedBy=len(df[df['movieId']==df_movie_pivot.index[indices.flatten()[i]]]['rating'])
        
        if len(movieName)==0: 
            ii -=1
            continue
        if i==0: #This is the 
            print("\n",n_neigh,"Movies similar to the movie", query_title,":\n")
        
        print("{0}: {1} ({2}),\n with cosine distance of {3:.3f}. ({4:.2f}% match)".format(ii+1,movieName,movieYear,distances.flatten()[i],match))
        print("Average Rating: {0:.3f}, Rated by {1} users.".format(avgRat,ratedBy))

In [ ]:
recommendation_movie()